네이버 영화 TOP 100 데이터 크롤링하기<br><br>
라이브러리 설치

In [1]:
# 링크 수집을 위한 라이브러리 설치
!pip install selenium

     |████████████████████████████████| 911kB 23.9MB/s 


In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas

In [3]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 k

분석할 데이터 모으기(크롤링)<br><br>
네이버 영화 TOP 100 페이지 별 링크 수집

In [4]:
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 실시간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 일간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 주간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 월간

def getPageLinksWantRange(startPageNo, lastPageNo):  # 중간에 끊어서 받고 싶을 때 ex) 1,2....2,3
  links = []    # 100개의 영화 링크를 수집할 리스트 변수 선언
  
  for pageNo in range(startPageNo -1, lastPageNo):
    # 실시간 TOP 100 영화 링크 수집
    url = "https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page=" +str(pageNo+1)

    req = requests.get(url) # knock knock~ 요청하고 req에 정보를 준다...페이지 접속
    soup = BeautifulSoup(req.text, 'lxml')  # html 파싱을 위한 객체 생성
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]') # 'div.lst_thum_wrap'에서부터 시작해줘 -> ul -> li -> a[href]
                                                                # ...<div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">
                                                                # soup는 공백을 > 로 해석
                                                                # driver는 > 로 알려줘야함.
    for movielink in movielinks:
      link = str(movielink.get('href'))
      links.append('https://series.naver.com' + link)   # 접속할 수 있는 전체 링크형태로 변환해서 append한다.
  return links

네이버 영화 제목, 평점, 장르, 줄거리, 개봉일 크롤링

In [5]:
def getMovieDataFromNaverSeries(links):
  title_infos = []    # 제목
  content_infos = []  # 줄거리
  genre_infos = []    # 장르
  score_infos = []    # 평점
  date_infos = []     # 개봉일

  url2 = 'https://www.naver.com'  # 속이기 위해 사람이 하는 것 처럼

  driver = webdriver.Chrome('chromedriver', options = options)
  driver.get(url2)
  time.sleep(3.0) # 3초 딜레이

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + "t") # 새 탭 열기

  for link in links:
    print(link + ' 수집 중......')  # 어디서 에러가 났는지 확인해보기 위해서
    driver.switch_to_window(driver.window_handles[-1])  # 새 탭을 열고 새 탭으로 활성화
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to_window(driver.window_handles[0])   # 새 창 활성화
    time.sleep(3.0)

    html_source = driver.page_source  # req.text 와 같은 동작
    html_soup = BeautifulSoup(html_source, 'lxml')

    # 청소년관람불가 영화 크롤링 할 경우 -> 인증 -> 수집 제외
    flag = html_soup.text[0:10]

    newflag = ''.join(flag)
    newflag = newflag.replace('\n', '')

    if newflag == '네이버':
      time.sleep(1.0)

      # 평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em')
      score = float(score.text)
      score = int(score)
      score_infos.append(score)
      print(score)

      # 장르 수집
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
      genre = genre.replace('장르', '')
      genre = genre.replace('\n', '')
      genre = genre.replace('\t', '')
      genre_infos.append(genre)

      # 제목, 줄거리
      try:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
      except:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')

      # 영화 상세보기 페이지로 이동
      movie_req = requests.get(movieInfoUrl)

      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find('meta', {'property':'og:title'}).get('content')  # body 부분이 아닌 head에 있어서 다른 방법
      title_infos.append(title)

      # 줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx') # soup는 공백을 줘도 되지만 안먹힐 때도 있어서 > 해준다.

      if len(contents_texts) == 0:
        content_infos.append('줄거리 오류')

      else:
        for contents in contents_texts:
          # 줄거리 데이터 클랜징 작업
          temp = contents.text
          temp = temp.replace('\r', '')    #줄바꿈 제거
          temp = temp.replace('\xa0', '')  #공백 제거
          content_infos.append(temp)

    elif newflag == '':
        print('청불 영화로 데이터 수집하지 않습니다.')
  print('수집 완료 합니다.....')
  print(len(score_infos),len(genre_infos),len(content_infos))
  driver.close()

  movie_dic = {'제목':title_infos, '평점':score_infos, '장르':genre_infos, '줄거리':content_infos}
  # 딕셔너리 -> 데이터프레임
  movie_df = pandas.DataFrame(movie_dic)

  # 수집된 정보 중에 중복 데이터 삭제
  movie_df2 = movie_df.drop_duplicates('줄거리', keep = 'first')
                               # 줄거리가 같으면 완전 같은 영화다... 제목은 동일한 제목의 다른 영화일 수 있다.
                               # keep = '' 은 첫 번째 수집한 값은 살린다.

  return movie_df2
    

csv 파일로 저장

In [6]:
def dftoCsv(movie_df,num):
  try:
    movie_df.to_csv(('movie_my_data_' +str(num)+ '.csv'),
                    sep = ',', na_rep='NaN', encoding = 'euc-kr') # 'utf-8'로 해도 되는데 한글 깨지면...'utf-8-sig' 
  except:
    print('Error')

크롤링 실행

In [7]:
# 1) 크롤링 할 링크 수집
links = getPageLinksWantRange(1, 5)

# 2) 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

# 3) csv 파일 저장
dftoCsv(movie_df2, len(movie_df2))

https://series.naver.com/movie/detail.nhn?productNo=6023769 수집 중......


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: use driver.switch_to.window instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: use driver.switch_to.window instead


9
https://series.naver.com/movie/detail.nhn?productNo=6291892 수집 중......
9
https://series.naver.com/movie/detail.nhn?productNo=6328803 수집 중......
6
https://series.naver.com/movie/detail.nhn?productNo=3400212 수집 중......
7
https://series.naver.com/movie/detail.nhn?productNo=6315371 수집 중......
9
https://series.naver.com/movie/detail.nhn?productNo=6243813 수집 중......
9
https://series.naver.com/movie/detail.nhn?productNo=4356726 수집 중......
9
https://series.naver.com/movie/detail.nhn?productNo=3293948 수집 중......
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=3241117 수집 중......
9
https://series.naver.com/movie/detail.nhn?productNo=6119777 수집 중......
7
https://series.naver.com/movie/detail.nhn?productNo=6139376 수집 중......
9
https://series.naver.com/movie/detail.nhn?productNo=3262026 수집 중......
9
https://series.naver.com/movie/detail.nhn?productNo=3024534 수집 중......
8
https://series.naver.com/movie/detail.nhn?productNo=6176542 수집 중......
8
https://series.naver.com/movi

크롤링한 TOP 100 영화 데이터 랭킹 시각화

In [8]:
import plotly.express as px
import plotly.graph_objects as go
import numpy
import pandas

In [10]:
df = pandas.read_csv('/content/movie_my_data_75_R.csv', encoding = 'euc-kr')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  75 non-null     int64 
 1   제목          75 non-null     object
 2   평점          75 non-null     int64 
 3   장르          75 non-null     object
 4   줄거리         75 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.1+ KB


In [11]:
fig = px.scatter(df, x = '제목', y = '평점', color = '장르')
fig.show()

In [13]:
df_score = df[df.평점 == 9]
df_score.tail()

,Unnamed: 0,제목,평점,장르,줄거리
66,76,업,9,액션,평생 모험을 꿈꿔 왔던 ‘칼’ 할아버지는 수천 개의 풍선을 매달아 집을 통째로 남아...
70,80,귀를 기울이면,9,애니메이션,"중학교 3학년 시즈쿠는 평소 책을 많이 읽는 소녀이다. 여름방학, 매번 도서카드에서..."
72,82,하울의 움직이는 성,9,애니메이션,"어느 날, 영문도 모른 채 마녀의 저주로 인해 할머니가 된 소녀 '소피'절망 속에서..."
73,83,이프 온리,9,코미디,"눈앞에서 사랑하는 연인을 잃은 남자는 다음 날 아침, 자신의 옆에서 자고 있는 연인..."
74,84,태극기 휘날리며,9,액션,가족의 생계를 책임지기 위해 열심히 살아가는 ‘진태’(장동건)는 약혼녀 ‘영신’(이...


9점 평점의 영화 중 가장 큰 비율을 차지하고 있는 장르는?

In [18]:
fig = px.pie(df_score, values = '평점', names = '장르')
fig.show()

일간 TOP 100 시각화

In [31]:
df2 = pandas.read_csv('/content/movie_my_data_61_D.csv', encoding = 'euc-kr')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  61 non-null     int64 
 1   제목          61 non-null     object
 2   평점          61 non-null     int64 
 3   장르          61 non-null     object
 4   줄거리         61 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.5+ KB


In [32]:
fig = px.bar(df2, x= '제목', y= '평점', color = '장르')
fig.show()

In [33]:
fig = px.pie(df2, values = '평점', names = '장르')
fig.show()

주간 TOP 100 시각화

In [34]:
df3 = pandas.read_csv('/content/movie_my_data_63_W.csv', encoding = 'euc-kr')
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  63 non-null     int64 
 1   제목          63 non-null     object
 2   평점          63 non-null     int64 
 3   장르          63 non-null     object
 4   줄거리         63 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.6+ KB


In [35]:
fig = px.bar(df3, x= '제목', y= '평점', color = '장르')
fig.show()

In [36]:
fig = px.pie(df3, values = '평점', names = '장르')
fig.show()

월간 TOP 100 시각화

In [37]:
df4 = pandas.read_csv('/content/movie_my_data_64_M.csv', encoding = 'euc-kr')
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  64 non-null     int64 
 1   제목          64 non-null     object
 2   평점          64 non-null     int64 
 3   장르          64 non-null     object
 4   줄거리         64 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.6+ KB


In [38]:
fig = px.bar(df4, x= '제목', y= '평점', color = '장르')
fig.show()

In [39]:
fig = px.pie(df4, values = '평점', names = '장르')
fig.show()